In [2]:
import math
import sympy as sp
import numpy as np


In [3]:

# ===== DEFINE SYMBOLIC VARIABLES =====
teta1, teta2, teta3 = sp.symbols('teta1 teta2 teta3')

# ===== FORWARD KINEMATICS =====
xf = 0.4*sp.cos(teta1) + 0.3*sp.cos(teta2) + 0.2*sp.cos(teta3)
yf = 0.4*sp.sin(teta1) + 0.3*sp.sin(teta2) + 0.2*sp.sin(teta3)

# Print symbolic expressions
print("Before Substitution Xf:", xf)
print("Before Substitution Yf:", yf)


Before Substitution Xf: 0.4*cos(teta1) + 0.3*cos(teta2) + 0.2*cos(teta3)
Before Substitution Yf: 0.4*sin(teta1) + 0.3*sin(teta2) + 0.2*sin(teta3)


In [4]:

# ===== SUBSTITUTE NUMERIC VALUES =====
res_xf = xf.subs([(teta1, math.pi/2), (teta2, math.pi/6), (teta3, math.pi/9)])
res_yf = yf.subs([(teta1, math.pi/2), (teta2, math.pi/6), (teta3, math.pi/9)])
print("\nAfter Substitution Xf:", res_xf.evalf())
print("After Substitution Yf:", res_yf.evalf())




After Substitution Xf: 0.447746145292513
After Substitution Yf: 0.618404028665134


In [6]:

# ===== ERROR FUNCTION (for future optimization) =====
# Define target X and Y (example)
X, Y = 0.8, 0.5
E = ((xf - X)**2 + (yf - Y)**2)/2
print("\nSymbolic Error function E:", E)



Symbolic Error function E: 0.125*(0.8*sin(teta1) + 0.6*sin(teta2) + 0.4*sin(teta3) - 1)**2 + 0.32*(0.5*cos(teta1) + 0.375*cos(teta2) + 0.25*cos(teta3) - 1)**2


In [7]:
!pip install symengine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 MB 15.7 MB/s eta 0:00:00


In [8]:
import symengine


In [9]:

# ===== JACOBIAN USING symengine =====
vars_sym = symengine.symbols('x y z')
f = symengine.sympify(['0.4*cos(x) + 0.3*cos(y) + 0.2*cos(z)',
                        '0.4*sin(x) + 0.3*sin(y) + 0.2*sin(z)'])
J = symengine.zeros(len(f), len(vars_sym))

for i, fi in enumerate(f):
    for j, s in enumerate(vars_sym):
        J[i, j] = symengine.diff(fi, s)

print("\nJacobian matrix J:")
print(J)




Jacobian matrix J:
[-0.4*sin(x), -0.3*sin(y), -0.2*sin(z)]
[0.4*cos(x), 0.3*cos(y), 0.2*cos(z)]



In [12]:
print(f)

[0.4*cos(x) + 0.3*cos(y) + 0.2*cos(z), 0.4*sin(x) + 0.3*sin(y) + 0.2*sin(z)]


In [13]:

# ===== TARGET POSITION =====
X_target = 0.8
Y_target = 0.5



In [14]:

# ===== ERROR FUNCTION =====
E = ((xf - X_target)**2 + (yf - Y_target)**2)/2
print("Symbolic Error function E:", E)



Symbolic Error function E: 0.125*(0.8*sin(teta1) + 0.6*sin(teta2) + 0.4*sin(teta3) - 1)**2 + 0.32*(0.5*cos(teta1) + 0.375*cos(teta2) + 0.25*cos(teta3) - 1)**2


In [15]:
# ===== GRADIENT =====
grad_E = [sp.diff(E, teta1), sp.diff(E, teta2), sp.diff(E, teta3)]
print("\nSymbolic Gradient:")
for i, g in enumerate(grad_E):
    print(f"dE/dtheta{i+1} =", g)




Symbolic Gradient:
dE/dtheta1 = 0.2*(0.8*sin(teta1) + 0.6*sin(teta2) + 0.4*sin(teta3) - 1)*cos(teta1) - 0.32*(0.5*cos(teta1) + 0.375*cos(teta2) + 0.25*cos(teta3) - 1)*sin(teta1)
dE/dtheta2 = 0.15*(0.8*sin(teta1) + 0.6*sin(teta2) + 0.4*sin(teta3) - 1)*cos(teta2) - 0.24*(0.5*cos(teta1) + 0.375*cos(teta2) + 0.25*cos(teta3) - 1)*sin(teta2)
dE/dtheta3 = 0.1*(0.8*sin(teta1) + 0.6*sin(teta2) + 0.4*sin(teta3) - 1)*cos(teta3) - 0.16*(0.5*cos(teta1) + 0.375*cos(teta2) + 0.25*cos(teta3) - 1)*sin(teta3)


In [16]:
# ===== GRADIENT DESCENT PARAMETERS =====
learning_rate = 0.1
max_iter = 100
teta_vals = np.array([math.pi/3, math.pi/3, math.pi/3])  # initial guess



In [17]:
# ===== GRADIENT DESCENT LOOP =====
for it in range(max_iter):
    # Evaluate gradient numerically at current angles
    grad_num = np.array([g.subs({teta1: teta_vals[0], teta2: teta_vals[1], teta3: teta_vals[2]})
                         for g in grad_E], dtype=float)

    # Update angles
    teta_vals = teta_vals - learning_rate * grad_num

    # Compute current error
    E_val = E.subs({teta1: teta_vals[0], teta2: teta_vals[1], teta3: teta_vals[2]}).evalf()

    if it % 10 == 0:  # print every 10 iterations
        print(f"Iteration {it}, E = {E_val}, angles = {teta_vals}")

print("\nFinal angles after Gradient Descent:", teta_vals)
xf_final = xf.subs({teta1: teta_vals[0], teta2: teta_vals[1], teta3: teta_vals[2]}).evalf()
yf_final = yf.subs({teta1: teta_vals[0], teta2: teta_vals[1], teta3: teta_vals[2]}).evalf()
print("Final end-effector position: X =", xf_final, "Y =", yf_final)


Iteration 0, E = 0.0946781411794593, angles = [1.02948474 1.03391294 1.03834114]
Iteration 10, E = 0.0528445679634576, angles = [0.87683414 0.91961375 0.96231243]
Iteration 20, E = 0.0293839663838881, angles = [0.76265597 0.83389139 0.90528568]
Iteration 30, E = 0.0164905536211508, angles = [0.67841434 0.77008035 0.86260197]
Iteration 40, E = 0.00947618253963268, angles = [0.61675576 0.7227354  0.83060545]
Iteration 50, E = 0.00567762027072239, angles = [0.57186852 0.68763361 0.80652046]
Iteration 60, E = 0.00362356950881186, angles = [0.53934158 0.66158505 0.78827436]
Iteration 70, E = 0.00251228378017565, angles = [0.51589124 0.64221253 0.77433236]
Iteration 80, E = 0.00190970125613628, angles = [0.49909515 0.62775688 0.7635625 ]
Iteration 90, E = 0.00158149989437564, angles = [0.48717362 0.61692097 0.75513159]

Final angles after Gradient Descent: [0.47952535 0.60946888 0.74903353]
Final end-effector position: X = 0.747340712969810 Y = 0.492459117719701
